In [ ]:
### SET WORKING DIR
# Any 'sar.txt' files in this dir will be processed
#dir = r'/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/performance/pods/perf-nineteen-0-6fbf77596d-ththg/sar-4_14_localsave'

sar_list1 = [
    {'name': 'kafka-broker', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/dummy/aibench-20190506184740525004/dummy-0-557689c859-t2nw8/perf-fmx123-0-cd4b4877c-bj8tp/sar-06052019185157'},
    {'name': 'kafka-broker', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/dummy/aibench-20190508102117526598/dummy-0-557689c859-zshj8/perf-fmx123-0-cd4b4877c-nsl2h/sar-08052019102507'},
    {'name': 'kafka-broker', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/dummy/aibench-20190508103214820439/dummy-0-557689c859-z6gtk/perf-fmx123-0-cd4b4877c-nsl2h/sar-08052019103234'},
    {'name': 'kafka-broker', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/dummy/aibench-20190508103344507186/dummy-0-557689c859-6plqb/perf-fmx123-0-cd4b4877c-nsl2h/sar-08052019103400'},
    {'name': 'kafka-broker', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/dummy/aibench-20190508103835204358/dummy-0-557689c859-cps5c/perf-fmx123-0-cd4b4877c-nsl2h/sar-08052019103849'},
    {'name': 'kafka-broker', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/dummy/aibench-20190508114752682806/dummy-0-557689c859-hkpsl/perf-fmx123-0-cd4b4877c-xczvf/sar-08052019115123'},
    {'name': 'kafka-broker', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/dummy/aibench-20190508115312008026/dummy-0-557689c859-nc6b7/perf-fmx123-0-cd4b4877c-lbq95/sar-08052019115627'},
]

sar_list2 = [
    {'name': 'kafka-broker',      'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/kafka-benchmark/aibench-20190529110121067216/e2e-kafkabroker-configsar-0-9786f6b65-np68c/perf-fmx055-0-8cd5fc48c-rwqrr/sar-29052019110230'},
    {'name': 'kafka-broker-one',  'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/kafka-benchmark/aibench-20190529110121067216/e2e-kafkabroker-one-configsar-0-77dc5dc4b5-f52w2/perf-fmx098-0-579cc784cf-296nb/sar-29052019110226'},
    {'name': 'kafka-broker-two',  'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/kafka-benchmark/aibench-20190529110121067216/e2e-kafkabroker-two-configsar-0-7fb5df7c9f-v6xmt/perf-fmx036-0-c64bd69df-4t2vh/sar-29052019110226'},
    {'name': 'kafka-consumer-01', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/kafka-benchmark/aibench-20190529110121067216/kafka-benchmark-consumer-01-configsar-0-55dbf9b6d5-mj47r/perf-fmx124-0-55b7cbd864-vdwth/sar-29052019110225'},
    {'name': 'kafka-consumer',    'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/kafka-benchmark/aibench-20190529110121067216/kafka-benchmark-consumer-configsar-0-6949455d86-crdj5/perf-fmx118-0-dbb97b579-vvgt5/sar-29052019110225'},
    {'name': 'kafka-producer',    'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/kafka-benchmark/aibench-20190529110121067216/kafka-benchmark-producer-configsar-0-68d8c85c8f-f2vvb/perf-fmx120-0-6ff46d8888-kz96w/sar-29052019110230'},
]

test_list = [{'name': 'srgans', 'dir': '/mnt/nvme_nfs_share/AIBench_shared_disk/Scenario/performance/pods/perf-nineteen-0-6fbf77596d-ththg/sar-4_14_localsave'}]

workload_list = test_list

In [ ]:
from datetime import datetime
from glob import glob
import numpy as np
import os
import pandas as pd
import time
import re
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell

pd.set_option('display.max_colwidth', -1)

In [ ]:
# 'cursor' state for where to position chart in worksheet
chart_position_state = {}

def add_chart(workbook, chartsheet, datasheet, df, chart_type, start_row, end_row, title, y_col_names,
              x_col_name=None, chart_subtype=None, x_label=None, y_label=None, x_log_base=None, y_log_base=None,
              x_min=None, x_max=None, y_min=None, y_max=None, disable_legend=False, additional_series=None,
              num_format='General', series_names=None, gap=None, x_interval_unit=None, y_interval_unit=None,
              x_interval_tick=None, y_interval_tick=None, series_filters=None, trendline=None,
              x_visible=True, y_visible=True, x_scale=2, y_scale=2, chart_position='below', series_creator=None):
    
    if workbook not in chart_position_state:
        chart_position_state[workbook] = {}
    if chartsheet not in chart_position_state[workbook]:
        chart_position_state[workbook][chartsheet] = {
            'chart_col' : 1,
            'chart_row_right' : 1,
            'chart_row_below' : 1,
        }
    
    # check columns exist
    columns = df.columns
    if x_col_name is None:
        x_col = None
        categories = None
    else:
        if not x_col_name in columns:
            #print("x-axis %s is not in columns: %s"%(x_col_name, columns))
            return False
        x_col = columns.get_loc(x_col_name) + 1
        categories = [datasheet, start_row, x_col, end_row, x_col]
    for y_col_name in y_col_names: 
        if not y_col_name in columns:
            #print("y-axis %s is not in columns: %s"%(y_col_name, columns))
            return False
    
    # create chart
    chart_worksheet = workbook.get_worksheet_by_name(chartsheet)
    chart = workbook.add_chart({'type': chart_type, 'subtype': chart_subtype})
    chart.show_blanks_as('span')
    
    
    # add main series
    for series_idx, y_col_name in enumerate(y_col_names):
        y_col = columns.get_loc(y_col_name) + 1
        if series_names is not None:
            series_name = series_names[series_idx]
        else:
            series_name = y_col_name

        if series_filters is None and series_creator is None:
            chart.add_series({
                'name':       series_name,
                'categories': categories,
                'values':     [datasheet, start_row, y_col, end_row, y_col],
                'gap':        gap,
                'trendline':  trendline,
            })
        else:
            if 'row' not in df.columns:
                print('ERROR: could not find \'row\' column in df')
                return False
            
            #find unique combinations, get row/col of each
            if series_filters is not None:
                filtered_df = filter_df(series_filters, df)
            else:
                filtered_df = df
                
            if series_creator is not None:
                series_names = filtered_df[series_creator].unique()
            else:
                series_names = [series_name]

            for series_name in series_names:
                if series_creator is not None:
                    series_df = filtered_df[filtered_df[series_creator] == series_name]
                else:
                    series_df = filtered_df
                
                values = '=('
                categories = '=('
                for row in series_df['row'].values:
                    values += datasheet + '!' + str(xl_rowcol_to_cell(row+1, y_col)) + ','
                    categories += datasheet + '!' + str(xl_rowcol_to_cell(row+1, x_col)) + ','
                values = values.rstrip(',') + ')'
                categories = categories.rstrip(',') + ')'

                chart.add_series({
                    'name':       series_name,
                    'categories': categories,
                    'values':     values,
                    'gap':        gap,
                    'trendline':  trendline,
                })
    
    
    # add additional series formatted by user
    # Could just chart raw values but xlsxwriter prints annoying warning so instead I'll write data to sheet and chart that
    if additional_series is not None:
        cat_count = val_count = 0
        for series in additional_series:
            chart_col = chart_position_state[workbook][chartsheet]['chart_col']
            chart_row = chart_position_state[workbook][chartsheet]['chart_row_below']
            cat_start_cell = xl_rowcol_to_cell(chart_row + cat_count, chart_col)
            val_start_cell = xl_rowcol_to_cell(chart_row + val_count, chart_col + 1)
            cat_count += len(series['categories'])
            val_count += len(series['values'])
            cat_end_cell = xl_rowcol_to_cell(chart_row + cat_count - 1, chart_col)
            val_end_cell = xl_rowcol_to_cell(chart_row + val_count - 1, chart_col + 1)
            chart_worksheet.write_column(cat_start_cell, series['categories'])
            chart_worksheet.write_column(val_start_cell, series['values'])
            series['categories'] = chartsheet + '!' + cat_start_cell + ':' + cat_end_cell
            series['values'] = chartsheet + '!' + val_start_cell + ':' + val_end_cell
            chart.add_series(series)
        
    # title
    chart.set_title({
        'name': title,
        'name_font': {'size': 18},
    })
    
    # legend
    chart.set_legend({
        'font': {'size': 12},
        'position': 'top',
        'none': disable_legend,
    })
    
    # axis
    chart.set_x_axis({
        'name': x_label,
        'name_font': {'size': 12},
        'num_font': {'size': 11},
        'label_position': 'low',
        'min': x_min,
        'max': x_max,
        'log_base': x_log_base,
        'num_format': num_format,
        'interval_unit': x_interval_unit,
        'interval_tick': x_interval_tick,
        'visible': x_visible,
    })
    chart.set_y_axis({
        'name': y_label,
        'name_font': {'size': 12},
        'num_font': {'size': 11},
        'label_position': 'low',
        'min': y_min,
        'max': y_max,
        'log_base': y_log_base,
        'num_format': num_format,
        'interval_unit': y_interval_unit,
        'interval_tick': y_interval_tick,
        'visible': y_visible,
    })
    
    # position & size
    if chart_position == 'below':  # anything other than 'below' will put to right
        chart_col = chart_position_state[workbook][chartsheet]['chart_col'] = 1  # reset column
        chart_row = chart_position_state[workbook][chartsheet]['chart_row_below']
    else:
        chart_col = chart_position_state[workbook][chartsheet]['chart_col']
        chart_row = chart_position_state[workbook][chartsheet]['chart_row_right']
        
    chart_worksheet.insert_chart(chart_row, chart_col, chart, {'x_scale': x_scale, 'y_scale': y_scale})

    chart_position_state[workbook][chartsheet]['chart_col'] = chart_col + (8 * x_scale)  # advance column
    chart_position_state[workbook][chartsheet]['chart_row_right'] = chart_row  # save row in case next chart is 'right'
    chart_position_state[workbook][chartsheet]['chart_row_below'] = chart_row + (15 * y_scale)  # advance row in case next chart is 'below'

    return True

In [ ]:
def filter_df(filters, df):
    filter_string = ""
    for column, values in filters.items():
        if column not in df.columns:
            return None
        else:
            if values is None:  # get all of them
                filter_vals = list(df[column].unique())
            elif type(values) is list:  # get list
                filter_vals = values
            else:  # get one value
                filter_vals = [values]

            if filter_string != "":
                filter_string += " & "
            filter_string += "(df['" + str(column) + "'].isin(" + str(filter_vals) + "))"

    filter_string = "df.loc[" + filter_string + "]"
    return eval(filter_string)

In [ ]:
# convert sar.txt to individual files
for entry in workload_list:
    workload = entry['name']
    sar_dir = entry['dir']
    file = sar_dir + '/sar.txt'
    outfile = sar_dir + "/" + workload + '_sar.xlsx'
    
    if not os.path.isfile(file):
        print("Can't find file %s"(file))
        continue
    
    print("")
    print("%s : %s"%(workload, file))
    
    with open(file) as fp:
        linenum = 0
        header = False
        filetype = None
        cpu_cores_stat = None
        #cpustat = None
        memstat = None
        netstat = None
        iostat = None
        for line in fp:
            linenum += 1
            
            if linenum % 5000 == 0:
                print("\tProcessed %d lines..."%(linenum))
            
            # line after blank is header
            if line.isspace() or line == "":
                header = True
            else:
                if header == True:
                    header = False
                    if re.search('%usr', line):
                        filetype = "cpu_cores_stat"
                        if cpu_cores_stat is None:
                            cpu_cores_stat = pd.DataFrame()
                            cpu_columns = line.split()
                            cpu_columns[0] = "Timestamp"
                            cpu_columns = pd.Series(cpu_columns)
                    elif re.search('kbmemfree', line):
                        filetype = "memstat"
                        if memstat is None:
                            memstat = pd.DataFrame()
                            mem_columns = line.split()
                            mem_columns[0] = "Timestamp"
                            mem_columns = pd.Series(mem_columns)
                    elif re.search('rd_sec/s', line):
                        filetype = "iostat"
                        if iostat is None:
                            iostat = pd.DataFrame()
                            io_columns = line.split()
                            io_columns[0] = "Timestamp"
                            io_columns = pd.Series(io_columns)
                    elif re.search('rxpck/s', line):
                        filetype = "netstat"
                        if netstat is None:
                            netstat = pd.DataFrame()
                            net_columns = line.split()
                            net_columns[0] = "Timestamp"
                            net_columns = pd.Series(net_columns)
                    else:
                        #ignore it
                        filetype = None
                else:
                    # add to dataframe
                    if filetype == "cpu_cores_stat":
                        cpu_cores_stat = cpu_cores_stat.append(pd.Series(line.split()), ignore_index=True)
                    elif filetype == "memstat":
                        memstat = memstat.append(pd.Series(line.split()), ignore_index=True)
                    elif filetype == "iostat":
                        iostat = iostat.append(pd.Series(line.split()), ignore_index=True)
                    elif filetype == "netstat":
                        netstat = netstat.append(pd.Series(line.split()), ignore_index=True)
        
        # set column headers
        cpu_cores_stat.columns = cpu_columns
        memstat.columns = mem_columns
        iostat.columns = io_columns
        netstat.columns = net_columns
        
        # create summary cpustat
        cpustat = cpu_cores_stat[cpu_cores_stat['CPU'] == 'all']
        cpustat = cpustat.reset_index(drop=True)
        
        #list of all dfs
        result_dfs = [cpu_cores_stat, cpustat, memstat, iostat, netstat]
        
        # add some columns to all
        FMT = '%H:%M:%S'
        for df in result_dfs:
            # convert to numeric
            cols = df.columns.drop('Timestamp')
            df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
            
            # row
            df['row'] = list(range(len(df.index)))  # because index may not be a number
            
            # duration
            df['duration'] = None
            duration_index = df.columns.get_loc("duration")
            for index, row in df.iterrows():
                if index+1 < len(df.index):
                    s1 = row['Timestamp']
                    s2 = df.iloc[index+1]['Timestamp']
                    tdelta = datetime.strptime(s2, FMT) - datetime.strptime(s1, FMT)
                    df.iloc[index, duration_index] = tdelta.seconds
            df.iloc[-1, duration_index] = df.iloc[-2]['duration']  # set last interval duration
            
            # cumulative duration
            df['cum_duration'] = df['duration'].cumsum()
        
        # add some columns to specific dfs
        netstat['rx+txkB/s'] = netstat['rxkB/s'] + netstat['txkB/s']
        
        #create xlsx writer and save data
        print("\tPrinting Excel stats")
        writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
        workbook = writer.book
        cpu_cores_stat.to_excel(writer, sheet_name='corestat')
        cpustat.to_excel(writer, sheet_name='cpustat')
        memstat.to_excel(writer, sheet_name='memstat')
        iostat.to_excel(writer, sheet_name='iostat')
        netstat.to_excel(writer, sheet_name='netstat')
        
        # create charts
        add_chart(
            workbook=workbook,
            chartsheet='cpustat',
            datasheet='cpustat',
            df=cpustat,
            chart_type='area',
            chart_subtype='stacked',
            start_row=1,
            end_row=len(cpustat.index),
            x_col_name='cum_duration',
            y_col_names=[
                '%iowait',
                '%usr',
                '%idle',
                '%sys',
            ],
            series_names=[
                'IO wait',
                'User',
                'Idle',
                'System',
            ],
            title='CPU Time',
            x_label='Time (s)',
            y_label='% Cycles Spent',
            y_min=0,
            y_max=100,
            num_format='0',
        )
        add_chart(
            workbook=workbook,
            chartsheet='memstat',
            datasheet='memstat',
            df=memstat,
            chart_type='scatter',
            chart_subtype='smooth',
            start_row=1,
            end_row=len(memstat.index),
            x_col_name='cum_duration',
            y_col_names=[
                '%memused',
            ],
            series_names=[
                'Footprint',
            ],
            title='DRAM Footprint',
            x_label='Time (s)',
            y_label='Memory Used (%)',
            num_format='0',
            disable_legend=True,
        )
        add_chart(
            workbook=workbook,
            chartsheet='iostat',
            datasheet='iostat',
            df=iostat,
            chart_type='scatter',
            chart_subtype='smooth',
            start_row=1,
            end_row=len(iostat.index),
            x_col_name='cum_duration',
            y_col_names=[
                '%util',
            ],
            title='IO Utilization per Device',
            x_label='Time (s)',
            y_label='Utilization (%)',
            num_format='0',
            series_creator='DEV'
        )
        add_chart(
            workbook=workbook,
            chartsheet='netstat',
            datasheet='netstat',
            df=netstat,
            chart_type='scatter',
            chart_subtype='smooth',
            start_row=1,
            end_row=len(netstat.index),
            x_col_name='cum_duration',
            y_col_names=[
                'rxkB/s',
            ],
            title='Network Read Bandwidth per Device',
            x_label='Time (s)',
            y_label='KB/s',
            num_format='0',
            series_creator='IFACE'
        )
        add_chart(
            workbook=workbook,
            chartsheet='netstat',
            datasheet='netstat',
            df=netstat,
            chart_type='scatter',
            chart_subtype='smooth',
            start_row=1,
            end_row=len(netstat.index),
            x_col_name='cum_duration',
            y_col_names=[
                'txkB/s',
            ],
            title='Network Write Bandwidth per Device',
            x_label='Time (s)',
            y_label='KB/s',
            num_format='0',
            series_creator='IFACE'
        )
        add_chart(
            workbook=workbook,
            chartsheet='netstat',
            datasheet='netstat',
            df=netstat,
            chart_type='scatter',
            chart_subtype='smooth',
            start_row=1,
            end_row=len(netstat.index),
            x_col_name='cum_duration',
            y_col_names=[
                'rx+txkB/s',
            ],
            title='Network Bandwidth per Device',
            x_label='Time (s)',
            y_label='KB/s',
            num_format='0',
            series_creator='IFACE'
        )
        
        writer.save()
        
        #print("\tPrinting cpu stats")
        #filename = sar_dir + "/" + workload + '_cpustat.csv'
        #cpu_cores_stat.to_csv(filename)

        #print("\tPrinting mem stats")
        #filename = sar_dir + "/" + workload + '_memstat.csv'
        #memstat.to_csv(filename)

        #print("\tPrinting io stats")
        #filename = sar_dir + "/" + workload + '_iostat.csv'
        #iostat.to_csv(filename)

        #print("\tPrinting net stats")
        #filename = sar_dir + "/" + workload + '_netstat.csv'
        #netstat.to_csv(filename)
        
        print("\tFinished file")
print("Done")